In [1]:
import pandas as pd 
import numpy as np 

In [2]:
data = pd.read_csv('Data/Reviews.csv')
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...


In [3]:
data.shape

(568454, 10)

Preprocessing
EDA

In [4]:
#Data Cleaning
# 1 . Handling null values
#gives count of null values against each column
data.isna().sum() 


Id                         0
ProductId                  0
UserId                     0
ProfileName               26
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [5]:
data['UserId'].nunique(dropna=True)

256059

In [6]:
total_rows = data.shape[0]
data.dropna(how = 'any', inplace=True)
new_row_count = data.shape[0]
print('removed_rows=',total_rows-new_row_count)

removed_rows= 53


In [7]:
#sorting data
sorted_data = data.sort_values('ProductId',axis = 0 , ascending = True ,na_position = 'last' )

In [8]:
# 3.removing redundant data 
#removing duplicates
sorted_data = sorted_data.drop_duplicates(subset = ['UserId'],keep='first')
non_redundant = sorted_data.drop_duplicates(subset = ['UserId','Time','ProfileName','Text'],keep='first')
# removing data where helpfulness numerator is smaller than helpfulness denominator
non_redundant_data = non_redundant[non_redundant["HelpfulnessNumerator"]<=non_redundant["HelpfulnessDenominator"]]

In [9]:
b = non_redundant_data.shape[0]
print('rows_removed=',new_row_count-b)

rows_removed= 312359


In [10]:
non_redundant_data.shape[0] 

256042

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
def pos_neg(x):
    if x >3: 
        return 1
    return 0

filtered_data = non_redundant_data['Score'].map(pos_neg)

In [13]:
non_redundant_data['Score'] = filtered_data

In [14]:
df = non_redundant_data
df.shape[0]

256042

In [15]:
pd.value_counts(df['Score'])

Score
1    199390
0     56652
Name: count, dtype: int64

In [16]:
df.isna().any()

Id                        False
ProductId                 False
UserId                    False
ProfileName               False
HelpfulnessNumerator      False
HelpfulnessDenominator    False
Score                     False
Time                      False
Summary                   False
Text                      False
dtype: bool

In [17]:
df.shape

(256042, 10)

Text Preprocessing

In [18]:
import re
from bs4 import BeautifulSoup

In [19]:
#text_preprocessing(i):
# i = re.sub(r"http\S+","",i)        --removes HTTP content
# i = re.sub(r"[^A-Za-z0-9]+"," ",i)   --removes special characters and punctuations
# i = re.sub('\S*\d\S*',"",i)         --removes alpha-numeric words
# i = BeautifulSoup(i).get_text()     --- to remove html content
    

In [20]:
#performing decontraction e.g. can't = can not
def decontraction(phrase):
    phrase = re.sub(r"won't","will not",phrase)
    phrase = re.sub(r"can't","can not",phrase)
    phrase = re.sub(r"doesn't","does not",phrase)
    phrase = re.sub(r"\'t","not",phrase)
    phrase = re.sub(r"\'d","would",phrase)
    phrase = re.sub(r"\'s"," is",phrase)
    phrase = re.sub(r"\'ve"," have",phrase)
    phrase = re.sub(r"\'m"," am",phrase)
    phrase = re.sub(r"\'re"," are",phrase)
    return phrase


In [21]:
#converting to lower case
sentence = ' '.join([i.lower() for i in sentence.split()])

NameError: name 'sentence' is not defined

In [ ]:
#removing stopwords
import nltk
from nltk.corpus import stopwords
# sentence = " ".join([i for i in sentence.split() if i not in nltk.stopwords('english')])
    

In [ ]:
#Stemming words
ps = nltk.PorterStemmer()

# sentence = " ".join([ps.stem(i) for i in sentence.split()])

In [ ]:
#progress bar
from tqdm import tqdm
import pdb

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


In [ ]:
corpus = []
for i in tqdm(df['Text'].values) :
    i = re.sub(r'http\S+','',str(i))                #removes HTTP content
    i = decontraction(i)
    i = re.sub(r"[^A-Za-z0-9]+"," ",str(i))    #removes special characters and punctuations
    i = re.sub('\S*\d\S*',"",i)         #removes alpha-numeric words
    i = BeautifulSoup(i).get_text()
    i = ' '.join([j.lower() for j in i.split()])
    i = " ".join([j for j in i.split() if j not in stopwords.words('english')])
    i = " ".join([ps.stem(j) for j in i.split()])
    corpus.append(i)
    
    
    
    

In [ ]:
corpus[100]

In [ ]:
len(corpus)

In [ ]:
corpus[0]

# Featurization

Featurization, also known as feature engineering, is the process of selecting, extracting, and transforming raw data into features that can be used as input to a machine learning model. The goal of featurization is to create a representation of the data that captures the most important information and relationships between the features and the target variable.
Featurization in NLP :
some of the methods are:
1. BOW
2. TF-IDF
3. word embedding